# Análisis de Sentimientos usando RNN

asdasdasd

In [3]:
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import numpy as np
from keras.datasets import imdb
np.random.seed(3)
(X_train, y_train), (X_test, y_test) = imdb.load_data(seed=15)

imdb.load_data(nb_words=3000, seed=15)
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

/usr/local/lib/python2.7/dist-packages/keras/datasets/imdb.py:44: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


In [4]:
#Embedding(top_words, embedding_vector_length, input_length=500)
print X_train.shape
print X_train

(25000, 500)
[[    0     0     0 ...,     4    86   273]
 [    0     0     0 ...,   705     9   150]
 [    0     0     0 ...,    44    12    32]
 ..., 
 [    0     0     0 ...,    22     8   377]
 [    0     0     0 ...,     4  2554   647]
 [    0     0     0 ..., 84643     4  5957]]


In [1]:
import os    
os.environ['THEANO_FLAGS'] = "device=gpu,floatX=float32,cuda.root=/usr/local/cuda-8.0/" 

import theano
import keras
#print(theano.config)

import matplotlib.pyplot as plt
from IPython.display import Image, display, SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, rmsprop
from keras.callbacks import LearningRateScheduler

from scipy.misc import imread
import cPickle as pickle
import numpy as np
import math

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
Using Theano backend.


Para el desarrollo de esta actividad se utilizará una GPU, utilizando un driver nvidia GeForce 940M, acelerando en una gran cantidad los cálculos realizados por la red convolucional.